In [8]:
import numpy as np 
import cv2

In [10]:
# cut out part of the video
#from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#ffmpeg_extract_subclip("./chessboard_videos/out13F.mp4", 35, 45, targetname="./chessboard_videos/out13F_cut.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [18]:
# import video 
cap = cv2.VideoCapture("./chessboard_videos/out2F_cut.mp4")

# get number of frames
amount_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
start_frame = 0

# save frames to array (take them somewhere from the middle of the video)
frames = []
gray_frames = []
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

for i in range(start_frame, start_frame + 50):  # use 50 frames
    ret, frame = cap.read()
    
    if not ret:
         print("Can't receive frame (stream end?). Exiting ...")
         break
     
    # convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    frames.append(frame)
    gray_frames.append(gray_frame)

cap.release()
cv2.destroyAllWindows() # destroy all opened windows

In [19]:
imS = cv2.resize(frames[0], (1280, 720)) 
cv2.imshow('test', imS)
cv2.waitKey(0)

cv2.destroyAllWindows()

50

In [41]:
# go through the frames and try to detect a checkerboard in them
corners_detected = {}  # frame_id, corners
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)  # todo: choose better params

chessboard_width = 5
chessboard_height = 7

# prepare object points
object_points = np.zeros((chessboard_width * chessboard_height, 3), np.float32)
object_points[:,:2] = np.mgrid[0:chessboard_height, 0:chessboard_width].T.reshape(-1,2)
print(object_points)
 
# Arrays to store object points and image points from all the images.
obj_points = [] # 3d point in real world space
img_points = [] # 2d points in image plane.

i = 0
while i < len(gray_frames):
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray_frames[i], (chessboard_height, chessboard_width), None)
    
    if ret:
        # If found, add object points, image points (after refining them)
        obj_points.append(object_points)
        
        corners_refined = cv2.cornerSubPix(gray_frames[i], corners, (11,11), (-1,-1), criteria)  # todo: choose better params
        corners_detected[i] = corners_refined
        
        img_points.append(corners_refined)
        
        i += 1
    else:
        i += 5  # skip the next five frames
    if len(corners_detected) >= 10:
        break

print(f'Number of frames where corners were detected: {len(corners_detected)}')

[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [5. 0. 0.]
 [6. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [5. 1. 0.]
 [6. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [5. 2. 0.]
 [6. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]
 [5. 3. 0.]
 [6. 3. 0.]
 [0. 4. 0.]
 [1. 4. 0.]
 [2. 4. 0.]
 [3. 4. 0.]
 [4. 4. 0.]
 [5. 4. 0.]
 [6. 4. 0.]]
Number of frames where corners were detected: 10


In [39]:
cv2.drawChessboardCorners(frames[5], (chessboard_height, chessboard_width), corners_detected[5], True)
imS = cv2.resize(frames[5], (1280, 720)) 
cv2.imshow('img', imS)
cv2.waitKey(0)
 
cv2.destroyAllWindows()

In [30]:
# calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray_frames[0].shape[::-1], None, None)
print(f'Camera Matrix: {mtx}')
print(f'Distortion: {dist}')

Camera Matrix: [[5.34198759e+03 0.00000000e+00 1.79078178e+03]
 [0.00000000e+00 5.11547089e+03 1.18833790e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion: [[-0.35981675  0.56206324 -0.02915499 -0.03313839 -1.28855586]]


In [40]:
# undistort an image - first refine camera matrix
img = frames[0]
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
print(f'New camera matrix: {newcameramtx}')

# undistort - option 1
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst_1 = dst[y:y+h, x:x+w]
cv2.imwrite('images/distorted.png', frames[0])
cv2.imwrite('images/undistorted-1.png', dst_1)

# undistort - option 2
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst_2 = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
 
# crop the image
x, y, w, h = roi
dst_2 = dst_2[y:y+h, x:x+w]
cv2.imwrite('images/undistorted-2.png', dst_2)

New camera matrix: [[5.07002532e+03 0.00000000e+00 1.69467856e+03]
 [0.00000000e+00 5.07438613e+03 1.16869786e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


True

In [38]:
# check reprojection error
mean_error = 0
for i in range(len(obj_points)):
 imgpoints2, _ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv2.norm(img_points[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(obj_points)) )

total error: 0.4143807207681351
